# Imagenet Evaluation Script
modified from [the evluation script by OpenAI](https://colab.research.google.com/github/openai/clip/blob/master/notebooks/Prompt_Engineering_for_ImageNet.ipynb).

In [ ]:
# !pip install -q -U jax jaxlib
# !pip install -q pandas
!pip install -q ipywidgets
!pip install -q -U flax
!pip install -q sentence-transformers
#!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q transformers
!pip install -q torch torchvision

     |████████████████████████████████| 184kB 8.7MB/s 
     |████████████████████████████████| 122kB 45.8MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 92kB 6.2MB/s 
     |████████████████████████████████| 2.5MB 35.9MB/s 
     |████████████████████████████████| 1.2MB 43.3MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 3.3MB 33.1MB/s 
     |████████████████████████████████| 901kB 34.3MB/s 
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.14 which is incompatible.


In [ ]:
import os 
import sys
import json

import numpy as np
import pandas as pd

os.environ['TOKENIZERS_PARALLELISM'] = "false"

import transformers
from transformers import AutoTokenizer

import torch
import torchvision
from torchvision import transforms
from torchvision.transforms import CenterCrop, ConvertImageDtype, Normalize, Resize, ToTensor
from torchvision.transforms.functional import InterpolationMode
from tqdm.notebook import tqdm

!wget -q -N https://github.com/huggingface/transformers/raw/master/examples/research_projects/jax-projects/hybrid_clip/modeling_hybrid_clip.py
!wget -q -N https://github.com/huggingface/transformers/raw/master/examples/research_projects/jax-projects/hybrid_clip/configuration_hybrid_clip.py

sys.path.append('.')

from modeling_hybrid_clip import FlaxHybridCLIP

# Choosing the model to evaluate

In [ ]:
# MODEL_TYPE = 'mClip'
MODEL_TYPE = 'clip_italian'

# Loading the model

In [ ]:
if MODEL_TYPE == 'mClip':
    from sentence_transformers import SentenceTransformer
    # Here we load the multilingual CLIP model. Note, this model can only encode text.
    # If you need embeddings for images, you must load the 'clip-ViT-B-32' model
    se_language_model = SentenceTransformer('clip-ViT-B-32-multilingual-v1')
    se_image_model = SentenceTransformer('clip-ViT-B-32')
    language_model = lambda queries: se_language_model.encode(queries, convert_to_tensor=True, show_progress_bar=False).cpu().detach().numpy()
    image_model = lambda images: se_image_model.encode(images, batch_size=1024, convert_to_tensor=True, show_progress_bar=False).cpu().detach().numpy()
elif MODEL_TYPE == 'clip_italian':
    import jax
    from jax import numpy as jnp
    TOKENIZER_NAME = "dbmdz/bert-base-italian-xxl-uncased"
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, cache_dir=None, use_fast=True)
    model = model = FlaxHybridCLIP.from_pretrained("clip-italian/clip-italian")
    def tokenize(texts):
        inputs = tokenizer(texts, max_length=96, padding="max_length", return_tensors="np")
        return inputs['input_ids'], inputs['attention_mask']

    language_model = lambda queries: np.asarray(model.get_text_features(*tokenize(queries)))
    image_model = lambda images: np.asarray(model.get_image_features(images.permute(0, 2, 3, 1).numpy(),))

INFO:absl:Starting the local TPU driver.
INFO:absl:Unable to initialize backend 'tpu_driver': Not found: Unable to find driver in registry given worker: local://


INFO:absl:Unable to initialize backend 'tpu': Invalid argument: TpuPlatform is not available.
Some weights of FlaxHybridCLIP were not initialized from the model checkpoint at clip-italian/clip-italian and are newly initialized: {('logit_scale',)}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Preparing the translated ImageNet labels

In [ ]:
!wget -N -q https://raw.githubusercontent.com/clip-italian/clip-italian/imagenet_templates/evaluation/imagenet_labels_IT.tsv
classes_df = pd.read_csv('./imagenet_labels_IT.tsv', sep='\t', header=0)
imagenet_classes = list(classes_df['query_short_translated'])  # list(classes_df['query_long_translated'])
imagenet_templates = ['{}']

print(f"{len(imagenet_classes)} classes, {len(imagenet_templates)} templates")

1000 classes, 1 templates


# Set up Validation Set

In [ ]:
val_preprocess = transforms.Compose([
    Resize([224], interpolation=InterpolationMode.BICUBIC),
    CenterCrop(224),
    ToTensor(),
    Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
])

In [ ]:
print('Downloading Imagenet validation set...')
!wget -N -q --show-progress https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
print('Downloading Imagenet devkit...')
!wget -N -q --show-progress https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz
print('Done.')

images = torchvision.datasets.ImageNet('./', split='val', transform=val_preprocess)
loader = torch.utils.data.DataLoader(
    images,
    batch_size=1024,
    shuffle=False,
    num_workers=2,
    persistent_workers=True,
    drop_last=False
)

ILSVRC2012_img_val. 100%[===================>]   6.28G  11.7MB/s    in 15m 14s 
ILSVRC2012_devkit_t 100%[===================>]   2.45M  8.95MB/s    in 0.3s    
Done.


# Creating zero-shot classifier weights

In [ ]:
def zeroshot_classifier(classnames, templates):
    zeroshot_weights = []
    for classname in tqdm(classnames):
        texts = [template.format(classname) for template in templates]
        class_embeddings = language_model(texts)
        class_embeddings = class_embeddings / np.linalg.norm(class_embeddings, axis=-1, keepdims=True)
        class_embedding = np.mean(class_embeddings, axis=0)
        class_embedding /= np.linalg.norm(class_embedding, axis=-1)
        zeroshot_weights.append(class_embedding)
    zeroshot_weights = np.stack(zeroshot_weights, axis=1)
    return zeroshot_weights


zeroshot_weights = zeroshot_classifier(imagenet_classes, imagenet_templates)

# Zero-shot prediction

In [ ]:
def accuracy(output, target, topk=(1,)):
    output = torch.from_numpy(np.asarray(output))
    target = torch.from_numpy(np.asarray(target))
    pred = output.topk(max(topk), dim=1, largest=True, sorted=True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]

In [ ]:
top_ns = [1, 5, 10, 100]
acc_counters = [0. for _ in top_ns]
n = 0.

for i, (images, target) in enumerate(tqdm(loader)):
    images = images
    target = target.numpy()
    # predict
    image_features = image_model(images)
    image_features = image_features / np.linalg.norm(image_features, axis=-1, keepdims=True)
    logits = 100. * image_features @ zeroshot_weights

    # measure accuracy
    accs = accuracy(logits, target, topk=top_ns)
    for j in range(len(top_ns)):
        acc_counters[j] += accs[j]
    n += images.shape[0]

tops = {f'top{top_ns[i]}': acc_counters[i] / n * 100 for i in range(len(top_ns))}

print(tops)


{'top1': 22.122, 'top5': 43.672, 'top10': 52.59, 'top100': 81.084}


OpenAI:  
    prompt engineering: {top1: 55.73, 'top5': 83.45}
  
mClip - multilanguage clip:  
    short translation:                      {'top1': 20.146, 'top5': 36.57, 'top10': 42.912, 'top100': 67.106}  
  
clip-italian:  
    short translation:                      {'top1': 22.122, 'top5': 43.672, 'top10': 52.59, 'top100': 81.084}  
    short translation + prompt engineering: {'top1': 21.886, 'top5': 43.086, 'top10': 51.739999999999995, 'top100': 82.06599999999999}  
    long tanslation:                        {'top1': 21.12, 'top5': 42.472, 'top10': 51.086, 'top100': 81.44}